In [1]:
import numpy as np
import math
import random

In [2]:
def generate_binary_array(num_centroids):
    # Determine k from num_centroids
    k = int(np.log2(num_centroids))

    # Generate all possible binary combinations of length k
    b = np.array([list(map(int, np.binary_repr(i, width=k))) for i in range(num_centroids)])

    return b

In [3]:
def channel_with_memory(num_level, epsilon, delta):
    Pr = np.zeros((num_level, num_level))
    n = int(np.log2(num_level))

    # Transition probability matrix for the binary symmetric channel with memory
    Pr_z = np.array([
        [(1 - epsilon + delta) / (1 + delta), epsilon / (1 + delta)],
        [(1 - epsilon) / (1 + delta), (epsilon + delta) / (1 + delta)]
    ])

    for x in range(num_level):
        for y in range(num_level):
            binary_x = np.array([int(bit) for bit in np.binary_repr(x, width=n)])
            binary_y = np.array([int(bit) for bit in np.binary_repr(y, width=n)])
            binary_z = binary_x ^ binary_y  # XOR operation

            if binary_z[0] == 1:
                probability = epsilon
            else:
                probability = 1 - epsilon
            for i in range(1, n):
                probability *= Pr_z[binary_z[i - 1], binary_z[i]]

            Pr[x, y] = probability

    return Pr

In [4]:
# Simulated Annealing Algorithm
def simulated_annealing(T_0, alpha, T_f, b, N_fail, N_success, N_cut, k, epsilon, delta, centroids, partitions, num_centroids):
    T = T_0
    count = 0
    count_success = 0
    count_fail = 0
    b_history = []

    prob_points = []

    # Loop over each partition
    for partition in partitions:
        # Calculate the probability of samples falling in this partition
        prob = len(partition) / 500000
        prob_points.append(prob)

    conditional_prob = channel_with_memory(num_centroids, epsilon, delta)

    while T > T_f and count_fail < N_fail:

        b_prime = random.sample(b, len(b))

        delta_Dc = 0

        distortion_b = 0
        distortion_b_prime = 0

        for g in range(0, num_centroids):
            for h in range(0, num_centroids):
                distortion_b = distortion_b + prob_points[g] * conditional_prob[h,b[g]] * ((centroids[b[g]]-centroids[h])**2)
                distortion_b_prime = distortion_b_prime + prob_points[g] * conditional_prob[h, b_prime[g]] * ((centroids[b_prime[g]]-centroids[h])**2)

        distortion_b = distortion_b * (1 / k)
        distortion_b_prime = distortion_b_prime * (1 / k)
        delta_Dc = distortion_b_prime - distortion_b
        b_history.append(distortion_b)

        if delta_Dc <= 0:
            b = b_prime
            count_success = count_success + 1
            count_fail = 0
        else:
            rand_num = random.uniform(0, 1)
            if rand_num <= math.exp(-delta_Dc / T):
                b = b_prime
            count_fail = count_fail + 1

        if count >= N_cut or count_success >= N_success:
            T = alpha * T
            count = 0
            count_success = 0
        count = count + 1

    return b

In [5]:
# Function to generate normalized source signal that will be used for training
def generate_source_signal(distribution, num_samples=500000):

    if distribution.lower() == 'laplace':
        source = np.random.laplace(loc=0, scale=np.sqrt(1/2), size=num_samples)
    else:
        source = np.random.normal(loc=0, scale=1, size=num_samples)

    # Normalize (zero-mean, unit variance)
    source = (source - np.mean(source)) / np.std(source)

    return source

In [6]:
def generate_initial_codebook(source, num_centroids):
    min_samples = np.min(source)
    max_samples = np.max(source)
    width = (max_samples - min_samples) / num_centroids
    centroids = []
    for i in range(num_centroids):
        # Calculate the current centroid
        centroid_current = min_samples + (i + 0.5) * width
        centroids.append(centroid_current)

    return centroids

In [7]:
def cosq_design(source, current_codebook, epsilon, b_obtained, tol=1e-4, max_iter=100):
    n_codewords = len(current_codebook)
    P_Y_given_X = channel_with_memory(n_codewords, epsilon, 10)

    print(P_Y_given_X)

    # Initialize codebook (ENSURE IT'S A NUMPY ARRAY)
    codebook = np.asarray(current_codebook.copy())  # Convert to NumPy array

    signal_power = np.mean(source ** 2)

    for iteration in range(max_iter):
        # --------------------------------------------------
        # Generalized NNC (Nearest Neighbor Condition)
        # --------------------------------------------------

        # Initialize the partitions
        partitions = [[] for _ in range(n_codewords)]

        for v in source:
          distortions = []

          # Iterate over each partition index
          for i in range(num_centroids):
            # Assuming v is assigned to i
            distortion = 0

            for j in range(num_centroids):
              # Compute the total distortion assuming that v is assigned to partition[i]
              distortion += P_Y_given_X[j, b_obtained[i]] * ((v - codebook[j])**2)

            distortions.append(distortion)

          # To find the minimum i from the distortion list
          min_distortion_idx = np.argmin(distortions)
          # Add v to that partition
          partitions[min_distortion_idx].append(v)

        # To visualize the partition of the line
        print(len(partitions[0]),len(partitions[4]),len(partitions[8]),len(partitions[12]))
        # --------------------------------------------------
        # Generalized CC (Centroid Condition)
        # --------------------------------------------------
        new_codebook = np.zeros_like(codebook)
        for i in range(n_codewords):
            numerator = 0.0
            denominator = 0.0

            for j in range(n_codewords):
                # If there is no element in that partition, skip that partition set
                if len(partitions[j]) == 0:
                    continue


                prob = P_Y_given_X[i, b_obtained[j]]
                sum_v = np.sum(partitions[j])
                count = len(partitions[j])

                numerator += prob * sum_v
                denominator += prob * count

            new_codebook[i] = numerator / denominator




        print(new_codebook)
        # Check convergence
        codebook_change = np.max(np.abs(new_codebook - codebook))
        codebook = new_codebook.copy()

        if codebook_change < tol:
            break


    # Calculate MSE
    summation = 0
    for i in range(n_codewords):
        for x in partitions[i]:
            for j in range(n_codewords):
                summation = summation + P_Y_given_X[b_obtained[i], j]*((x - codebook[j])**2)
    
    summation = summation / len(sampled_source)
    
    snr = 10 * np.log10(signal_power / summation)

    return codebook, partitions, snr

In [8]:
#--------------------------------------------------------------
# Parameter declarations
#--------------------------------------------------------------

num_centroids = 16
# For SA
T_0 = 10
alpha = 0.97
T_f = 0.00025
N_fail = 50000
N_success = 5
N_cut = 200
k = 10
# For channel
delta = 10

initial_distribution = 'Gaussian'

sampled_source = generate_source_signal(initial_distribution)

codebooks_set = []

# Create an empty array b with length num_centroids
init_b = [0] * num_centroids

# Populate the array with values from 0 to num_centroids - 1
for i in range(num_centroids):
    init_b[i] = i


In [9]:
# COSQ for noiseless. Works for a preset low noise epsilon
init_codebook = generate_initial_codebook(sampled_source, num_centroids)
codebooks_set.append(np.array(init_codebook))

print(init_codebook)
noiseless_codebook, noiseless_partition, noiseless_snr = cosq_design(sampled_source, codebooks_set[-1],  1e-11, init_b)
print(noiseless_snr)
codebooks_set.append(np.array(noiseless_codebook))

[-4.338430025348626, -3.7832984303312713, -3.2281668353139166, -2.673035240296562, -2.117903645279207, -1.5627720502618523, -1.007640455244498, -0.45250886022714276, 0.10262273479021111, 0.6577543298075659, 1.2128859248249206, 1.7680175198422754, 2.32314911485963, 2.878280709876985, 3.433412304894339, 3.9885438999116944]
[[1.00000000e+00 9.09090909e-13 8.26446281e-14 8.26446281e-13
  8.26446281e-14 7.51314801e-26 7.51314801e-14 7.51314801e-13
  9.09090909e-13 8.26446281e-25 7.51314801e-26 7.51314801e-25
  8.26446281e-13 7.51314801e-25 7.51314801e-13 7.51314801e-12]
 [9.09090909e-13 1.00000000e+00 8.26446281e-13 8.26446281e-14
  7.51314801e-26 8.26446281e-14 7.51314801e-13 7.51314801e-14
  8.26446281e-25 9.09090909e-13 7.51314801e-25 7.51314801e-26
  7.51314801e-25 8.26446281e-13 7.51314801e-12 7.51314801e-13]
 [8.26446281e-14 8.26446281e-13 1.00000000e+00 9.09090909e-13
  7.51314801e-14 7.51314801e-13 8.26446281e-14 7.51314801e-26
  7.51314801e-26 7.51314801e-25 9.09090909e-13 8.264462

In [10]:
# Once we obtain the noiseless_codebook, we perform SA algorithm to obtain the shuffling before we train with noisy channels
b_from_sa = simulated_annealing(T_0, alpha, T_f, init_b, N_fail, N_success, N_cut, k, 0.005, delta, noiseless_codebook, noiseless_partition, num_centroids)

print(b_from_sa)

[15, 10, 6, 9, 7, 4, 14, 3, 13, 12, 11, 5, 2, 1, 8, 0]


In [11]:
# Starting with noiseless codebook and b obtained from SA algorithm, run the training for epsilon = 0.005
codebook_1, partition_1, snr_1 = cosq_design(sampled_source, codebooks_set[-1], 0.005, b_from_sa)
print(snr_1)
codebooks_set.append(np.array(codebook_1))

[[9.93643798e-01 4.51861664e-04 4.08915285e-05 4.11175620e-04
  4.08915285e-05 1.85955109e-08 3.72096173e-05 3.74152986e-04
  4.51861664e-04 2.05485068e-07 1.85955109e-08 1.86983002e-07
  4.11175620e-04 1.86983002e-07 3.74152986e-04 3.76221168e-03]
 [4.51861664e-04 9.93643798e-01 4.11175620e-04 4.08915285e-05
  1.85955109e-08 4.08915285e-05 3.74152986e-04 3.72096173e-05
  2.05485068e-07 4.51861664e-04 1.86983002e-07 1.85955109e-08
  1.86983002e-07 4.11175620e-04 3.76221168e-03 3.74152986e-04]
 [4.08915285e-05 4.11175620e-04 9.93643798e-01 4.51861664e-04
  3.72096173e-05 3.74152986e-04 4.08915285e-05 1.85955109e-08
  1.85955109e-08 1.86983002e-07 4.51861664e-04 2.05485068e-07
  3.74152986e-04 3.76221168e-03 4.11175620e-04 1.86983002e-07]
 [4.11175620e-04 4.08915285e-05 4.51861664e-04 9.93643798e-01
  3.74152986e-04 3.72096173e-05 1.85955109e-08 4.08915285e-05
  1.86983002e-07 1.85955109e-08 2.05485068e-07 4.51861664e-04
  3.76221168e-03 3.74152986e-04 1.86983002e-07 4.11175620e-04]
 [4.

In [12]:
codebook_2, partition_2, snr_2 = cosq_design(sampled_source, codebooks_set[-1], 0.01, b_from_sa)
print(snr_2)
codebooks_set.append(np.array(codebook_2))

[[9.87302454e-01 8.98364380e-04 8.09263636e-05 8.18255455e-04
  8.09263636e-05 7.36363636e-08 7.37100000e-05 7.45290000e-04
  8.98364380e-04 8.17438017e-07 7.36363636e-08 7.44545455e-07
  8.18255455e-04 7.44545455e-07 7.45290000e-04 7.53571000e-03]
 [8.98364380e-04 9.87302454e-01 8.18255455e-04 8.09263636e-05
  7.36363636e-08 8.09263636e-05 7.45290000e-04 7.37100000e-05
  8.17438017e-07 8.98364380e-04 7.44545455e-07 7.36363636e-08
  7.44545455e-07 8.18255455e-04 7.53571000e-03 7.45290000e-04]
 [8.09263636e-05 8.18255455e-04 9.87302454e-01 8.98364380e-04
  7.37100000e-05 7.45290000e-04 8.09263636e-05 7.36363636e-08
  7.36363636e-08 7.44545455e-07 8.98364380e-04 8.17438017e-07
  7.45290000e-04 7.53571000e-03 8.18255455e-04 7.44545455e-07]
 [8.18255455e-04 8.09263636e-05 8.98364380e-04 9.87302454e-01
  7.45290000e-04 7.37100000e-05 7.36363636e-08 8.09263636e-05
  7.44545455e-07 7.36363636e-08 8.17438017e-07 8.98364380e-04
  7.53571000e-03 7.45290000e-04 7.44545455e-07 8.18255455e-04]
 [8.

In [13]:
codebook_3, partition_3, snr_3 = cosq_design(sampled_source, codebooks_set[-1], 0.05, b_from_sa)
print(snr_3)
codebooks_set.append(np.array(codebook_3))


[[9.37104250e-01 4.27901484e-03 3.71238730e-04 3.92731499e-03
  3.71238730e-04 1.69515402e-06 3.40725958e-04 3.60452198e-03
  4.27901484e-03 1.95388805e-05 1.69515402e-06 1.79329452e-05
  3.92731499e-03 1.79329452e-05 3.60452198e-03 3.81320483e-02]
 [4.27901484e-03 9.37104250e-01 3.92731499e-03 3.71238730e-04
  1.69515402e-06 3.71238730e-04 3.60452198e-03 3.40725958e-04
  1.95388805e-05 4.27901484e-03 1.79329452e-05 1.69515402e-06
  1.79329452e-05 3.92731499e-03 3.81320483e-02 3.60452198e-03]
 [3.71238730e-04 3.92731499e-03 9.37104250e-01 4.27901484e-03
  3.40725958e-04 3.60452198e-03 3.71238730e-04 1.69515402e-06
  1.69515402e-06 1.79329452e-05 4.27901484e-03 1.95388805e-05
  3.60452198e-03 3.81320483e-02 3.92731499e-03 1.79329452e-05]
 [3.92731499e-03 3.71238730e-04 4.27901484e-03 9.37104250e-01
  3.60452198e-03 3.40725958e-04 1.69515402e-06 3.71238730e-04
  1.79329452e-05 1.69515402e-06 1.95388805e-05 4.27901484e-03
  3.81320483e-02 3.60452198e-03 1.79329452e-05 3.92731499e-03]
 [3.

In [14]:
codebook_4, partition_4, snr_4 = cosq_design(sampled_source, codebooks_set[-1], 0.1, b_from_sa)
print(snr_4)
codebooks_set.append(np.array(codebook_4))

[[8.75677010e-01 8.03373403e-03 6.63335838e-04 7.44410218e-03
  6.63335838e-04 6.08564989e-06 6.14650639e-04 6.89774606e-03
  8.03373403e-03 7.37039820e-05 6.08564989e-06 6.82945154e-05
  7.44410218e-03 6.82945154e-05 6.89774606e-03 7.74080391e-02]
 [8.03373403e-03 8.75677010e-01 7.44410218e-03 6.63335838e-04
  6.08564989e-06 6.63335838e-04 6.89774606e-03 6.14650639e-04
  7.37039820e-05 8.03373403e-03 6.82945154e-05 6.08564989e-06
  6.82945154e-05 7.44410218e-03 7.74080391e-02 6.89774606e-03]
 [6.63335838e-04 7.44410218e-03 8.75677010e-01 8.03373403e-03
  6.14650639e-04 6.89774606e-03 6.63335838e-04 6.08564989e-06
  6.08564989e-06 6.82945154e-05 8.03373403e-03 7.37039820e-05
  6.89774606e-03 7.74080391e-02 7.44410218e-03 6.82945154e-05]
 [7.44410218e-03 6.63335838e-04 8.03373403e-03 8.75677010e-01
  6.89774606e-03 6.14650639e-04 6.08564989e-06 6.63335838e-04
  6.82945154e-05 6.08564989e-06 7.37039820e-05 8.03373403e-03
  7.74080391e-02 6.89774606e-03 6.82945154e-05 7.44410218e-03]
 [6.

In [15]:
codebook_5, partition_5, snr_5 = cosq_design(sampled_source, codebooks_set[-1], 0.05, b_from_sa)
print(snr_5)
codebooks_set.append(np.array(codebook_5))

[[9.37104250e-01 4.27901484e-03 3.71238730e-04 3.92731499e-03
  3.71238730e-04 1.69515402e-06 3.40725958e-04 3.60452198e-03
  4.27901484e-03 1.95388805e-05 1.69515402e-06 1.79329452e-05
  3.92731499e-03 1.79329452e-05 3.60452198e-03 3.81320483e-02]
 [4.27901484e-03 9.37104250e-01 3.92731499e-03 3.71238730e-04
  1.69515402e-06 3.71238730e-04 3.60452198e-03 3.40725958e-04
  1.95388805e-05 4.27901484e-03 1.79329452e-05 1.69515402e-06
  1.79329452e-05 3.92731499e-03 3.81320483e-02 3.60452198e-03]
 [3.71238730e-04 3.92731499e-03 9.37104250e-01 4.27901484e-03
  3.40725958e-04 3.60452198e-03 3.71238730e-04 1.69515402e-06
  1.69515402e-06 1.79329452e-05 4.27901484e-03 1.95388805e-05
  3.60452198e-03 3.81320483e-02 3.92731499e-03 1.79329452e-05]
 [3.92731499e-03 3.71238730e-04 4.27901484e-03 9.37104250e-01
  3.60452198e-03 3.40725958e-04 1.69515402e-06 3.71238730e-04
  1.79329452e-05 1.69515402e-06 1.95388805e-05 4.27901484e-03
  3.81320483e-02 3.60452198e-03 1.79329452e-05 3.92731499e-03]
 [3.

In [16]:
codebook_6, partition_6, snr_6 = cosq_design(sampled_source, codebooks_set[-1], 0.01, b_from_sa)
print(snr_6)
codebooks_set.append(np.array(codebook_6))

[[9.87302454e-01 8.98364380e-04 8.09263636e-05 8.18255455e-04
  8.09263636e-05 7.36363636e-08 7.37100000e-05 7.45290000e-04
  8.98364380e-04 8.17438017e-07 7.36363636e-08 7.44545455e-07
  8.18255455e-04 7.44545455e-07 7.45290000e-04 7.53571000e-03]
 [8.98364380e-04 9.87302454e-01 8.18255455e-04 8.09263636e-05
  7.36363636e-08 8.09263636e-05 7.45290000e-04 7.37100000e-05
  8.17438017e-07 8.98364380e-04 7.44545455e-07 7.36363636e-08
  7.44545455e-07 8.18255455e-04 7.53571000e-03 7.45290000e-04]
 [8.09263636e-05 8.18255455e-04 9.87302454e-01 8.98364380e-04
  7.37100000e-05 7.45290000e-04 8.09263636e-05 7.36363636e-08
  7.36363636e-08 7.44545455e-07 8.98364380e-04 8.17438017e-07
  7.45290000e-04 7.53571000e-03 8.18255455e-04 7.44545455e-07]
 [8.18255455e-04 8.09263636e-05 8.98364380e-04 9.87302454e-01
  7.45290000e-04 7.37100000e-05 7.36363636e-08 8.09263636e-05
  7.44545455e-07 7.36363636e-08 8.17438017e-07 8.98364380e-04
  7.53571000e-03 7.45290000e-04 7.44545455e-07 8.18255455e-04]
 [8.

In [17]:
codebook_7, partition_7, snr_7 = cosq_design(sampled_source, codebooks_set[-1], 0.005, b_from_sa)
print(snr_7)
codebooks_set.append(np.array(codebook_7))

[[9.93643798e-01 4.51861664e-04 4.08915285e-05 4.11175620e-04
  4.08915285e-05 1.85955109e-08 3.72096173e-05 3.74152986e-04
  4.51861664e-04 2.05485068e-07 1.85955109e-08 1.86983002e-07
  4.11175620e-04 1.86983002e-07 3.74152986e-04 3.76221168e-03]
 [4.51861664e-04 9.93643798e-01 4.11175620e-04 4.08915285e-05
  1.85955109e-08 4.08915285e-05 3.74152986e-04 3.72096173e-05
  2.05485068e-07 4.51861664e-04 1.86983002e-07 1.85955109e-08
  1.86983002e-07 4.11175620e-04 3.76221168e-03 3.74152986e-04]
 [4.08915285e-05 4.11175620e-04 9.93643798e-01 4.51861664e-04
  3.72096173e-05 3.74152986e-04 4.08915285e-05 1.85955109e-08
  1.85955109e-08 1.86983002e-07 4.51861664e-04 2.05485068e-07
  3.74152986e-04 3.76221168e-03 4.11175620e-04 1.86983002e-07]
 [4.11175620e-04 4.08915285e-05 4.51861664e-04 9.93643798e-01
  3.74152986e-04 3.72096173e-05 1.85955109e-08 4.08915285e-05
  1.86983002e-07 1.85955109e-08 2.05485068e-07 4.51861664e-04
  3.76221168e-03 3.74152986e-04 1.86983002e-07 4.11175620e-04]
 [4.

In [2]:
# Check partition set
for i in range(num_centroids):
    print(np.max(partittion_4[i]))
    print(np.mmin(partittion_4[i]))

NameError: name 'num_centroids' is not defined